In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("mushrooms.csv")
df.head(10)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
le = LabelEncoder()
ds = df.apply(le.fit_transform)

In [6]:
data = ds.values

In [7]:
X, y = data[:, 1:], data[:, 0]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
a = np.array([0, 5, 5, 1, 1, 1, 0, 1])

In [10]:
def prior_prob(y_train, label):
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train == label)

    return (class_examples)/float(total_examples)

In [12]:
y = np.array([0, 5, 5, 1, 1, 1, 1, 0, 0, 0])
prior_prob(y, 0)

0.4

In [13]:
def cond_prob(x_train, y_train, feature_col, feature_val, label):
    x_filtered = x_train[y_train==label]
    numerator = np.sum(x_filtered[:, feature_col]==feature_val)
    denominator = np.sum(y_train == label)
    return numerator/float(denominator)

In [14]:
def predict(x_train, y_train, Xtest):
    classes = np.unique(y_train)
    n_features = x_train.shape[1]
    post_probs = []
    for label in classes:
        likelihood = 1.0
        for f in range(n_features):
            cond = cond_prob(x_train, y_train, f, Xtest[f], label)
            likelihood*=cond
        prior = prior_prob(y_train, label)
        post = likelihood*prior
        post_probs.append(post)
    pred = np.argmax(post_probs)
    return pred

In [15]:
output = predict(X_train, y_train, X_test[1])
print(output)
print(y_test[1])

0
0


In [18]:
def score(X_train, y_train, X_test, y_test):
    pred = []
    for i in range(X_test.shape[0]):
        pred_label = predict(X_train, y_train, X_test[i])
        pred.append(pred_label)
    pred = np.array(pred)
    accuracy = np.sum(pred==y_test)/y_test.shape[0]
    return accuracy

In [19]:
print(score(X_train, y_train, X_test, y_test))

0.9944615384615385
